In [ ]:
%load_ext pycodestyle_magic

In [ ]:
import pandas as pd
import nltk, json
from IPython.display import JSON

nltk.download('wordnet', quiet=True)
nltk.download("stopwords", quiet=True)

REVIEW = "review"

# use smaller set
with open("cafe/reviews.json") as IN:
    data = pd.DataFrame(json.load(IN))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[REVIEW],
                                                    data['rating'],
                                                    stratify=data['rating'],
                                                    random_state=21)

JSON([len(x_train) + len(y_train), len(x_test) + len(y_test)])

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import HashingVectorizer

# TODO: preprocess data?

# pick vectorizer
vectorizer = HashingVectorizer()
x_train_features = vectorizer.fit_transform(x_train)
x_test_features = vectorizer.transform(x_test)

def train(classifier, x, y):
    classifier.fit(x, y)

# pick classifier
classifier = DummyClassifier(strategy="uniform")

# train
train(classifier,x_train_features, y_train)

In [ ]:
%%pycodestyle
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import HashingVectorizer

# TODO: preprocess data?

# pick vectorizer
vectorizer = HashingVectorizer()
x_train_features = vectorizer.fit_transform(x_train)
x_test_features = vectorizer.transform(x_test)

def train(classifier, x, y):
    classifier.fit(x, y)

# pick classifier
classifier = DummyClassifier(strategy="uniform")

# train
train(classifier,x_train_features, y_train)

In [ ]:
y_pred_test = classifier.predict(x_test_features)
y_proba = classifier.predict_proba(x_test_features)

from sklearn.metrics import f1_score
from IPython.display import JSON
output = []
x_test_list = x_test.to_list()
y_test_list = y_test.to_list()
experiment_size = y_pred_test.size // 5
trial = 0
experiment = []
y_true = []
y_pred = []
f1_scores = []
for i in range(y_pred_test.size):
    if trial == experiment_size and i + experiment_size <= y_pred_test.size:
        output.append(experiment)
        experiment = []
        trial = 0
        f1_scores.append(f1_score(y_true, y_pred))
        y_true = []
        y_pred = []
    y_true.append(y_test_list[i])
    y_pred.append(y_pred_test[i])
    experiment.append({
        "inputText": x_test_list[i],
        "realLabel": y_test_list[i],
        "classifierLabel": y_pred_test[i],
        "confidence": max(y_proba[i])
    })
    trial += 1

JSON(output)